In [16]:
import pandas as pd
import os, sys
import numpy as np
  
def const_func():

    # %% - Importing packages
    import numpy as np
    import math

    # %% - Defining constant function and class

    # defining round_half function
    def round_half(number):
        return round(number * 2.0) / 2.0

    class Constant(object):
        pass
    
    const = Constant()  

# %% Abaqus detail

    # defining file path to launcher bat on PC for command-line running of Abaqus
    const.abq = "C:\\Simulia\\CAE\\2018\\win_b64\\resources\\install\\cae\\launcher.bat"

# %% - Geometry

    # radius of cylinder, mm
    const.r = 300
    
    # circumference of cylinder, mm
    const.c = 2 * math.pi * const.r

    const.l = 1040

# %% - Node definition

    # fixing number of nodes
    const.node = 31000

    # need const.node = 68000 for 5.2% of designs that have high number of n_i combinations

    # defining bounds (nodes +- bounds %)
    const.bound = 0.02

    # find average mesh size
    const.avg_m = round_half(math.sqrt((const.l*const.c)/const.node))

# %% - Material information

    # # All material information is from MTM 49-3 data sheet. Using MTM49-3/T800 column, page 4 of Solvay data sheet.
    # # Taken E2 as the same ratio between T1000 fibre. 55% fibre volume fraction.
    #
    # Material name:
    # const.materialName = "MTM49-3/T800"
    
    # ply thickness, mm
    # const.plyT = 0.13
    
    # material constants:
    # MPa
    # const.E1 = 122000
    # const.E2 = const.E1*0.06                # same ratio of T1000 fibre
    
    # -
    # const.nu12 = 0.31                       #
    # const.nu13 = const.nu12                 #
    # const.nu23 = 0.45                       # standard nu23 value is used
    # const.nu21 = const.nu12*const.E2 / const.E1
    
    # MPa
    # const.G12 = 4900                        # assumed value
    # const.G13 = 4900                        # assumed value
    # const.G23 = 3230
    
    # physical constants
    # g/cm3
    # const.rho = 1.54                        # not sure if this is correct...

    # All material information is from Wichita data sheet for various different fibre volume fractions. If needing an
    # accurate model, need accurate data! All data points are for 70F (21 C), actual testing may be at a considerably
    # lower temperature during winter in the UK...

    # # Material name:
    const.materialName = "IM7/8552"

    # # ply thickness, mm
    const.plyT = 1.048/48
    # const.plyT = 0.14672

    # # material constants:
    # # MPa
    const.E1 = 138171  # (61.09 % Vf)
    const.E2 = 9721.6  # (60.38 % Vf)

    # # -
    const.nu12 = 0.356  # (61.09 % Vf)
    const.nu21 = const.E2*const.nu12 / const.E1   # (61.09 % Vf)
    const.nu23 = 0.450  # assumed value

    # # MPa
    const.G12 = 4688.4  # (58.93 % Vf)
    const.G13 = const.G12  # (58.93 % Vf) - assuming G12 = G13
    const.G23 = const.E2/(2*(1 + const.nu23))  # - assuming nu23 = 0.45...

    # # physical constants
    # # kg/mm3
    const.rho = 0.00000157

    # # Tsai-Wu values for IM7/8552 - taken from IM7/8552 property sheet // NASA (Overview of Coupon Testing of an
    # # IM7/8552 Composite Required to Characterize High-Energy Impact Dynamic Material Models - NASA/TM—2020-220498) //
    # # Cui et al (A comparison of failure criteria to predict delamination of unidirectional glass/epoxy specimens
    # # waisted through the thickness)

    # tensile strength in fibre direction, MPa [Hexcel]
    const.Xt = 2724

    # compressive strength in fibre direction, MPa [Hexcel]
    const.Xc = 1690

    # tensile strength in transverse direction, MPa [Hexcel]
    const.Yt = 64

    # compressive strength in transverse direction, MPa [NASA]
    const.Yc = 286

    # shear strength in XY plane, MPa [Hexcel]
    const.S = 120

    # cross-product term coefficient [Cui]
    const.f = -0.5

# %% - Layup information

    # laminate thickness, mm
    # const.h = const.noPly * const.plyT

    # layup as defined by the design choice
    # const.layup = ['X', '-X', 'X', '-X', '-X', 'X', '-X', 'X']

    # generating linearly space list of each ply number
    # const.plyList = np.linspace(1, const.noPly, const.noPly).tolist()

# %% - End of script

    return const
    
    
# preamble function (checking if folder exists and if .fil file is present) and define panda print options
def preamble(path_file):

    pd.set_option('display.max_rows', 1000)
    pd.set_option('display.max_columns', 1000)
    pd.set_option('display.width', 1000)    
    
    # creates path if it does not exist
    if not os.path.exists(path_file):
        # attempt to create it. If fails because of FilExistsError, pass
        try:
            os.makedirs(path_file)
        except FileExistsError:
            pass
        
    sys.path.append(os.getcwd())

    # changes current directory to path
    os.chdir(path_file)

    # replacing underscores with dashes as delineation between phi, t0, t1 and n is with underscores
    if '_' in path_file:
        path_file.replace('_', '-')      
    
# define laminate data frame based on inputs
def laminate(imp, load, pth, T0, T1,
             man='c', cit=1, nply=8,
             sym=True, des=None, N=None, ph=None, fib=None,
             obd=True, Pot=[False, 0, 1.0, 1.0], BC=False, rot=[0, 0],
             FOSM=False, ImpData=False,StepInfo=[1e-3],old_geom=False,dz='1.5'):   
    
    '''
    Inputs :>
            
        imp = string: p, r, s, a, b, c, d, e, f, g, h, i, j, m
            imperfection signature chosen, p = perfect, r = realistic (must define node file)
            
        man = string: 'i' or 'c' (typically c = current)
            manufacturing technique chosen to define how the variation in thickness is described
            
        load = string: (combinations of) b, l, n, t, v, d, s
                loading parameter(s) to say what steps to do in the analysis
            
        cit = int: 1 or [1, ..., x]
            citizen number if more than one analysis is taking place
            
        nply = 8 or int: [1, ..., x]
            number of plies
            
        des = None or [1, 2, 3, 4, 5, 6, 7, 8]
            design of cylinder (based on Composite Structures paper)
            
        N = None or [n_1, n_1, n_2, n_2]
            periodicity: number of T0 -> T1 -> T0 cycles in k-th layer
            
        ph = None or [phi_1, phi_1, phi_2, phi_2]
            direction of shearing for k-th layer
            
        T0 = None or [T0_1, -T0_1, T0_2, -T0_2]
            initial direction of shearing
            
        T1 = None or [T1_1, -T1_1, T1_2, -T1_2]
            final directino of shearing (at the centre of the period)
            
        sym = True or False
            whether the laminate is symmetric or not
            
        fib = None or string: s (straight), v (variable)
            array of fibre definitions
            
        Pot = [False or True, height of potting, size of potting in axial direction, eta factor]
            array to define whether potting occurs and size/impact of potting
            
        BC = False or True
            boolean to define whether BlueCrystal is to be used
            
        rot = [0, 0] or [th_y, th_z]
            rotation array to describe the y- and z-rotations (i.e. eccentric loading)
            
        FOSM = False or True
            boolean to define whether a FOSM analysis is taking place
            
        ImpData = list with: [string, best-fit radius]
            string to define where to find 'realistic' imperfection file (containing FourierModes)
            best-fit radius (from Fourier analysis)
            
        StepInfo = list with [Min Step Size] 
            default value is 1e-3. Min Step Size used within nonlinear steps
    '''

    import math
    import scipy.integrate as integrate
     
    def des_def(d, t01, t02, t11, t12):
    
        if d == 1:
            
            phi_list = [0]*8
            n_list = [8]*8
            
            t0_list = [t01, t01, t02, t02, t02, t02, t01, t01]
            t1_list = [t11, t11, t12, t12, t12, t12, t11, t11]
        
            fibre_list = ['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v']          
        
        elif d == 2:
            
            phi_list = [90]*8
            n_list = [12]*8
            
            t0_list = [t01, t01, t02, t02, t02, t02, t01, t01]
            t1_list = [t11, t11, t12, t12, t12, t12, t11, t11]
        
            fibre_list = ['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v']          
        
        elif d == 3:
            
            phi_list = [0]*8
            n_list = [0, 0, 8, 8, 8, 8, 0, 0]
            
            t0_list = [45, -45, t02, t02, t02, t02, -45, 45]
            t1_list = [45, -45, t12, t12, t12, t12,- 45, 45]
        
            fibre_list = ['s', 's', 'v', 'v', 'v', 'v', 's', 's']
        
        elif d == 4:
            
            phi_list = [0, 0, 90, 90, 90, 90, 0]
            n_list = [0, 0, 12, 12, 12, 12, 0, 0]
            
            t0_list = [45, -45, t02, t02, t02, t02, -45, 45]
            t1_list = [45, -45, t12, t12, t12, t12,- 45, 45]
        
            fibre_list = ['s', 's', 'v', 'v', 'v', 'v', 's', 's']
        
        elif d == 5:
            
            phi_list = [0]*8
            n_list = [8, 8, 0, 0, 0, 0, 8, 8]
            
            t0_list = [t01, t01, 0, 90, 90, 0, t01, t01]
            t1_list = [t11, t11, 0, 90, 90, 0, t11, t11]
        
            fibre_list = ['v', 'v', 's', 's', 's', 's', 'v', 'v']
                          
        elif d == 6:
            
            phi_list = [90, 90, 0, 0, 0, 0, 90, 90]
            n_list = [12, 12, 0, 0, 0, 0, 12, 12]
            
            t0_list = [t01, t01, 0, 90, 90, 0, t01, t01]
            t1_list = [t11, t11, 0, 90, 90, 0, t11, t11]
        
            fibre_list = ['v', 'v', 's', 's', 's', 's', 'v', 'v']
         
        elif d == 7:
            
            phi_list = [0, 0, 90, 90, 90, 90, 0, 0]
            n_list = [8, 8, 12, 12, 12, 12, 8, 8]
            
            t0_list = [t01, t01, t02, t02, t02, t02, t01, t01]
            t1_list = [t11, t11, t12, t12, t12, t12, t11, t11]
        
            fibre_list = ['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v']                    
         
        elif d == 8:
            
            phi_list = [90, 90, 0, 0, 0, 0, 90, 90]
            n_list = [12, 12, 8, 8, 8, 8, 12, 12]          
            
            t0_list = [t01, t01, t02, t02, t02, t02, t01, t01]
            t1_list = [t11, t11, t12, t12, t12, t12, t11, t11]
        
            fibre_list = ['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v']                     
        
        return n_list, phi_list, t0_list, t1_list, fibre_list
   
    def avgT(fibre, man, t0, t1, n, const):
    
        # defining sec function for integration
        def u(x):
            return 1. / math.cos(x)

        # calculating maximum shearing angle, integral of that shearing angle
        # and area of a ply that is sheared by the maximum amount, per type of
        # ply (Phi_1, Phi_2)

        # pre-allocating A_ply as empty list
        t_ply = []
        
        for k, f in enumerate(fibre):
            
            if man in {'c'}:
            
                # if f is straight (i.e. no shearing) then result is the ply thickness
                if f in {'s'}:
                    
                    t_ply.append(const.plyT)
        
                # if f is variable (i.e. shearing) then result is calcualted
                elif f in {'v'}:
                    
                    t_diff = t1[k] - t0[k]
                    
                    if n[k] == 0:
                        t_ply.append(const.plyT/math.cos(abs(math.pi*t0[k]/180)))
                    elif t_diff == 0 and t1[k] == t0[k] == 0:
                        t_ply.append(const.plyT)
                    elif t_diff == 0 and abs(t1[k]) > 0 and abs(t0[k]) > 0:
                        t_ply.append(const.plyT/math.cos(abs(math.pi*t1[k]/180)))
                    else:
                        t_diff_rad = t_diff * math.pi / 180
                        i = integrate.quad(u, math.pi * t0[k] / 180, math.pi * t1[k] / 180)
                        t_ply.append(const.plyT * abs(i[0]) / abs(t_diff_rad))
                    
            elif man in {'i'}:
                
                # if f is straight (i.e. no shearing) then result is the ply thickness
                if f in {'s'}:
                    
                    t_ply.append(const.plyT)
        
                # if f is variable (i.e. shearing) then result is calcualted
                elif f in {'v'}:
                    
                    t_diff = abs(t1[k] - t0[k])
                    
                    if n[k] == 0: # periodicity == 0
                        t_ply.append(const.plyT)
                    elif t_diff == 0 and t1[k] == t0[k] == 0:
                        t_ply.append(const.plyT)
                    elif t_diff == 0 and abs(t1[k]) > 0 and abs(t0[k]) > 0:
                        t_ply.append(const.plyT)
                    else:
                        t_diff_rad = t_diff * math.pi / 180
                        i = integrate.quad(u, 0, t_diff_rad)
                        t_ply.append(const.plyT * abs(i[0]) / abs(t_diff_rad))
        
        return sum(t_ply)
   
    def mass_calc(t_avg, const):
        
        # calculating area by summing plies and multiplying by 2 pi r
        # (assuming j = 1 and laminate of form
        A = 2*math.pi*const.r*t_avg

        # calculating mass (m = desity x volume = rho * area * length * 1000)
        # Multiply by 1000 for change unit from kg to g
        return round(A*const.rho*const.l*1000)
   
    # % first part of code checks for known exceptions

    # initial checks on variables
    if imp == 'r':
        if not ImpData:
            raise Exception('Imperfection chosen is realistic but ImpData has been defined')
        if len(ImpData) != 2:
            raise Exception(f'ImpData does not have 2 elements. Length of ImpData = {len(ImpData)}')
        if not os.path.exists(ImpData[0]):
            raise Exception(f'NodeFile chosen but cannot find file at path: {ImpData[0]}')
        if type(ImpData[1]) is not int and type(ImpData[1]) is not float:
            raise Exception(f'ImpData[1] is not an integer or float. ImpData[1] = {ImpData[1]}, type = {type(ImpData[1])}')

    # % second part of code defines design, periodicity (n), phi, t0, t1 and fibre.
    
    # if des variable is None (i.e. design has not been chosen) define laminate based on N, Ph, T0, T1 and fib
    if not des:
                
        if N is None:
            raise Exception('N has not been defined properly')
        if ph is None:
            raise Exception('ph has not been defined properly')
        if T0 is None:
            raise Exception('T0 has not been defined properly')
        if T1 is None:
            raise Exception('T1 has not been defined properly')
        if fib is None:
            raise Exception('fib has not been defined properly')
            
        if sym:
            
            # n, phi, t0, t1, fibre definition
            n = N + N[::-1]
            phi = ph + ph[::-1]
            t0 = T0 + T0[::-1]
            t1 = T1 + T1[::-1]            
            fibre = [fib]*nply
            
        else:
            
            # n, phi, t0, t1, fibre definition
            n = N
            phi = ph
            t0 = T0
            t1 = T1
            fibre = [fib]*len(t0)

    # if des is NOT None (i.e. design has been chosen = 1, 2, 3, 4, 5, 6, 7, 8))
    elif des:
        
        # n, phi, t0, t1, fibre definition
        n, phi, t0, t1, fibre = des_def(des, T0[0], T0[2], T1[0], T1[2])
        
    else:
        
        raise Exception('Variable `des` is incorrect.')
        
    # third part of code deals with imperfections, manufacturing techniques, citizens and 
        
    # check imp is within range of values
    if imp in {'p', 'r', 's', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'm'}:
        # p = perfect analysis (no imperfections) + linear buckling analysis + 1 mm compression
        # r = realistic analysis (imperfections from a measured cylinder) + nonlinear static
        # s = pSuedo random analysis (random imperfections up to wall thickness) + nonlinear static
        # [a, b, c, d, e] = backwards difference in FOSM analysis
        # [f, g, h, i, j] = forwards difference in FOSM analysis
        # m = mean imperfection signature in FOSM analysis
        pass
    else:
        raise Exception('Value `imp` is not correct')
        
    # potting check (potting list must include height of potting, number of elements in potting, size of element and eta, the stiffening factor)
    if not Pot[0]:
        pass
    elif len(Pot) != 4:
            raise Exception(f'Pot variable is not correct length (length = {len(Pot)})')
    elif type(Pot[0]) == bool and type(Pot[1]) == int and type(Pot[2]) == int and type(Pot[3]) == float:
        if Pot[1] % Pot[2] == 0:
            pass
        else:
            raise Exception(f'Pot axial mesh height ({Pot[0][2]}) does not go into Pot height'\
                            f' ({Pot[0][1]}) evenly.')
    else:
        raise Exception('Pot variable not properly defined.\n'\
                        f'Pot[0] should be True/False, Pot[0] = {Pot[0]}, type = {type(Pot[0])}'
                        f'Pot[1] should be int, Pot[1] = {Pot[1]}, type = {type(Pot[1])}'
                        f'Pot[2] should be int, Pot[2] = {Pot[2]}, type = {type(Pot[2])}'
                        f'Pot[3] should be float, Pot[3] = {Pot[3]}, type = {type(Pot[3])}')
        
    # if BlueCrystal is to be used (i.e. want sequential, integer file names)
    if BC:
        fileName = f'{cit}'
        
    # if not, use normal system
    elif not BC:
        if des:
            fileName = f'{cit}{man}_{load}_{imp}_Des{des}{t0[0]}{t1[0]}'
        elif not des:        
            fileName = f'{cit}{man}_{load}_{imp}_{phi[0]}_{t0[0]}_{t1[0]}_{n[0]}__{phi[2]}_{t0[2]}_{t1[2]}_{n[2]}'
    else:
        raise Exception('Variable `BC` incorrect')
        
    if Pot[0]:
        fileName += '_P'
        
    if rot[0] != 0 or rot[1] != 0:
        fileName += '_R'
        
    # full path is the path + file name + .inp
    fullPath = f'{pth}/{fileName}.inp'
    
    debug = 0
    
    if debug:
        
        print(des)
        print(man)
        print(cit)
        print(fullPath)
        print(fileName)
        print(imp)
        print(nply)
        print(phi)
        print(t0)
        print(t1)
        print(n)
        print(fibre)
        print(BC)
        print(Pot)
        print(rot)
        print(FOSM)
    
    # put data into Pandas DataFrame as a dictionary    
    df = pd.DataFrame({'Des': des, 'Man': man, 'Load':load, 'Cit': cit, 'FullPath': fullPath,
                           'FileName': fileName, 'Imp': imp, 'Nply': nply, 'Sym':sym, 'obd':obd,
                           'phi': [phi], 't0': [t0], 't1': [t1], 'n': [n], 'Fibre': [fibre],
                           'BC': BC, 'Pot': [Pot], 'Rot': [rot], 'FOSM': FOSM, 'ImpData': [ImpData], 'StepInfo':[StepInfo],
                           'OldGeom':[old_geom],'dz':[dz]})
    
    if df.OldGeom[0]:

        from const_designs import const_des
        const = const_des()
        
    else:
    
        from const import const_func
        const = const_func()

    # calculate average thickness
    df['t_avg'] = avgT(fibre, man, t0, t1, n, const)
    
    # calculate mass
    df['Mass'] = mass_calc(df.t_avg, const)

    return df

# convert data frame into tuple for multiprocessing
def tuplify(pop_data):

    '''defining pre processing function that takes information in pandas data frame and converts to a tuple to
    multiprocessing'''    
    
    # getting column data
    data_col = pop_data.columns.tolist()

    # putting other data into list
    data_inp = pop_data.values.tolist()

    # appending column data to other data
    data_inp = [[data_col] + [d] for d in data_inp]

    # putting nested list into list of tuples for input into run function
    tuple_pop = [tuple(d) for d in data_inp]

    return tuple_pop

# function to run tupled data frame
def run(df_tup):
        
    # convert tuple back into list and then dataframe
    df_list = list(df_tup)
    df = pd.DataFrame(df_list, columns=df_list.pop(0))
    
    if df.OldGeom[0]:

        from const_designs import const_des
        const = const_des()
        
    else:
    
        from const import const_func
        const = const_func()
        
    # defining Abaqus path
    abq = r"C:/Simulia/CAE/2018/win_b64/resources/install/cae/launcher.bat"

    # load functions
    from input_funcs.result import result_func
    from input_funcs.inp_main import inp
            
    if debug:
        
        inp_write = 1
        
        if inp_write:
        
            # running job and file creation script
            inp(df)
                
        return result_func(df, debug, const)
    
    else:
                    
        # running job and file creation script
        inp(df)
        
        # if .dat file exists (i.e. run has already been carried out) skip the run section)
        if os.path.exists(f'{df.FileName[0]}.dat'):
            
            pass
     
        # else if the .dat file has not been created, run the .inp file
        elif not os.path.exists(f'{df.FileName[0]}.dat'):
    
            # running analysis with job name and file name for analysis
            os.system(f"{abq} job={df.FileName[0]} interactive input={df.FileName[0]}.inp")
            # os.system(f"abaqus job={df.FileName[0]} interactive input={df.FileName[0]}.inp")
            # pass
        
        # if df.obd is false, delete obd file
        if not df.obd[0]:
            
            if os.path.exists(fr'{df.FullPath[0][:-3]}'+'odb'):
                      
                # remove obd file (as it is large)
                os.remove(fr'{df.FullPath[0][:-3]}'+'odb')

        return result_func(df, debug, const)
    
    
def lhs_lams(lam_type, cyl, n_var):

    # defining function to take LHS (n_samples-by-n_variables matrix of value 0 -> 1) into useable designs
    def lhs_to_lamina(lhs_in, cyl, lam_type):

        import numpy as np

        # lhs_in = unity-normalised LHS
        # cyl = [radius, length, thickness]
        # lam_type = 1 (RTS) or lam_type = 0 (SF)

        # return = lamina = [[phi1, t01, t11, n1, phi2, t02, t12, n2]*n_var]

        # define sub-function that takes phi and returns n_max based on geometry
        def phi_to_n_max(cyl, phi):

            # cyl = [radius, length, thickness]
            # phi = 0 (axial) or = 90 (circumferential)

            # assume MSR (minimum steering radius) is 50, therefore 1 period is 100 mm (minimum)
            msr = 100

            if phi == 0:
                n_max = np.floor(cyl[1]//msr) + 1
            elif phi == 90:
                n_max = np.floor(2*np.pi*cyl[0]//msr) + 1
            else:
                SyntaxError('Phi is not defined as 0 or 90 degree')

            return n_max

        lamina = np.zeros((np.shape(lhs_in)))

        # if lam_type = 1 (i.e. RTS laminate)
        if lam_type:

            for i, l in enumerate(lhs_in):
                lamina[i, 0] = round(l[0])*90 # defining phi_1
                lamina[i, 1] = round(l[1]*70) # defining t0_1
                lamina[i, 2] = round(l[2]*70) # defining t1_1
                lamina[i, 3] = int(l[3]*phi_to_n_max(cyl, lamina[i,0])) # defining n_1
                lamina[i, 4] = round(l[4])*90 # defining phi_2
                lamina[i, 5] = round(l[5]*70) # defining t0_2
                lamina[i, 6] = round(l[6]*70) # defining t1_2
                lamina[i, 7] = int(l[7]*phi_to_n_max(cyl, lamina[i,4])) # defining n_2

        # if lam_type = 0 (i.e. SF laminate)
        elif not lam_type:

            for i, l in enumerate(lhs_in):
                lamina[i, 0] = round(l[0]*90) # defining alpha_1
                lamina[i, 1] = round(l[1]*90) # defining alpha_2

        return np.asarray(lamina)

    ## Rapid tow sheared (RTS) laminates
    # number of RTS variables: [pm phi_1 < T0_1 | T1_1 > n_1 , pm phi_2 < T0_2 | T1_2 > n_2]s
#     n_rts_var = 8

    # defining number of samples 
    n_sam = 10*n_var
    # the number of samples is based on the 'levels' of each variable:
    # # https://stats.stackexchange.com/questions/58201/how-to-determine-the-sample-size-of-a-latin-hypercube-sampling
    # SMT use 10 * n dimensions. We have 8 dimensions, therefore 80 samples. Could change... Probably need to infill after initial 

    from pyDOE2 import lhs

    # call LHS from pyDOE2 for n_variables
    lhs_norm  = lhs(n_var, n_sam)

    # cyl is a vector with [radius, length, thickness] in mm of the cylinder in question
    # cyl is needed to define max periodicty if RTS laminate is chosen
#     cyl = [300, 1040, 1.05]

    # transform unity lhs to laminate definition for RTS (lam_type = 1)
    lams = lhs_to_lamina(lhs_norm, cyl, 1)

    return lams

# %% main file and function running
global debug, BC, cd
debug, BC, cd = 0, 0, os.getcwd()

import pandas as pd

if __name__ == '__main__':
        
    # define current directory (where the code will run from)
    cd = os.getcwd()
    
    # change current directory to directory as defined above
    os.chdir(cd)
    
    # define path to save results to
    master_path = r'C:\Users\rl14205\OneDrive - University of Bristol\Postdoc Research\SurrogateModelling\Results'

    # run checks on save path and run preamble code
    preamble(master_path)
    
    # call lhs_lam function
    lhs = lhs_lams(1, [300, 1040, 1.05], 8)

    # create empty data frame
    pop_in = pd.DataFrame()
    
    for x in lhs:
        
        phi = [int(x[0]), int(x[4])]
        t0 = [int(x[1]), int(x[5])]
        t1 = [int(x[2]), int(x[6])]
        n = [int(x[3]), int(x[7])]
        
        # add laminates to pop_in
        pop_in = pop_in.append(laminate(cit=1, imp='p', load='n', pth=master_path,
                                        T0=[t0[0],  -t0[0], t0[1],  -t0[1]],
                                        T1=[t1[0],  -t1[0], t1[1],  -t1[1]],
                                        N =[ n[0],    n[0],  n[1],    n[1]],
                                        ph=[phi[0], phi[0], phi[1], phi[1]],
                                        sym=True, fib='v'), ignore_index=True)
        
    print(lam)

    # pre-process pop_out and turn into tuple for multiprocessing
    pop_tuple = tuplify(pop_in)

    # define empty data frame
    pop_out = pd.DataFrame()
    
    # run pop_tuple through Parallel, output result to pop_out
    from joblib import Parallel, delayed
#     pop_out = pop_out.append(Parallel(n_jobs=5)(delayed(run)(pop) for pop in pop_tuple),ignore_index=True)
    
    # run in sequence
    # for p in pop_tuple:
        # pop_out = pop_out.append(run(p), ignore_index=True)
        
        # exit()
        
    os.chdir(cd)

    


ModuleNotFoundError: No module named 'const'